In [24]:
import pandas as pd
import modules #직접 만든 모듈파일
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [14]:
df = pd.read_csv('../../data/go_wvs.csv')

In [19]:
import numpy as np

# Word2Vec - KMeans

In [94]:
from sklearn.cluster import KMeans
cluster_count=20
model = KMeans(n_clusters=cluster_count)

In [95]:
x = df.T

In [96]:
x.shape

(2278, 300)

In [97]:
model.fit(x)
model.predict(x)
Y=x.copy()
Y['kmeans_id'] = model.predict(x)

In [98]:
km = list(Y['kmeans_id'])
max_count = 0
max_count_id = ''
for i in range (0, cluster_count):
    if km.count(i) > max_count:
        max_count = km.count(i);
        max_count_id = i;
    print(i,': ', km.count(i))
max_count_id

0 :  77
1 :  94
2 :  98
3 :  34
4 :  3
5 :  184
6 :  186
7 :  92
8 :  119
9 :  388
10 :  80
11 :  182
12 :  154
13 :  99
14 :  126
15 :  86
16 :  40
17 :  101
18 :  44
19 :  91


9

In [99]:
km_df = pd.DataFrame()
km_df['group_'+str(max_count_id)] = pd.Series(list(Y[Y['kmeans_id'] == max_count_id].index))
for i in range(0, cluster_count):
    col_name = 'group_'+str(i)
    words = list(Y[Y['kmeans_id'] == i].index)
    km_df[col_name] = pd.Series(words)

In [100]:
km_df

,group_9,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19
0,single,favorite,good,handsome,auxiliary,national,small,rich,expensive,interior,updated,unkind,Good,sleepy,different,exciting,prepared,democratic,Chinese,comfortable
1,more.,generous,bad,comic,marine,regional,ambiguous,environmental,burdensome,distinctive,digital,dirty,Free,nearby,active,perfect,unlikely,peaceful,Indian,scared
2,free,sincere,busy,unassuming,nuclear,sub,wide,unique,convenient,spacious,offline,rubbish,Next,dusty,available,great,enough,social,foreign,happy
3,twostory,smile,quiet,chatty,military,NaN,ordinary,visual,cheap,beautiful,unmanaged,tasteless,Luminous,underdeveloped,possible,nice,able,martial,Japanese,upset
4,personal,precious,friendly,funny,civil,NaN,blind,human,fun,fancy,online,irritating,Precious,scenic,secret,neat,want,incense,Asian,relaxed
5,sorry.,blessed,aggressive,calm,naval,NaN,absolute,diverse,simple,architectural,web,rude,Emotional,clogged,designated,improbable,forced,religious,English,embarrassed
6,own,joyful,blunt,sweet,admiral,NaN,raw,organic,easy,colorful,interactive,unconvincing,Happy,noisy,limited,famous,failed,temple,Greek,uncomfortable
7,time.,emotional,rusty,loud,civilian,NaN,actual,empowered,pleasant,restful,inline,illogical,Perfect,remote,mandatory,historic,waiting,pious,British,satisfied
8,senior,cherish,sloppy,polite,fire,NaN,extreme,strategic,difficult,trendy,searchable,ridiculous,Famous,central,fit,awesome,willing,Buddhist,Russian,excited
9,general,patriotic,big,cute,diplomatic,NaN,large,professional,popular,luxurious,website,wrong,Historic,outside,representative,sublime,wish,historical,European,careful


In [56]:
#3
group_ls = [1,3,13,14,15,17,20,21,24,27,33,40,41,42,44,46,57,73,79,83,84,86,89,93,94,97,99]
stop_words = modules.make_stop_words_ls(km_df, group_ls)
df = modules.remove_stop_words(df, stop_words)

In [57]:
#after 3
df.shape #2746 > #2278

(300, 2278)

In [47]:
#2
group_ls = [18,22,28,35,36,41,42,43,46,47,48,49,53,54,56,61,68,69,70,71,72,73,74,77,78,79,85,89,96,]
stop_words = modules.make_stop_words_ls(km_df, group_ls)
df = modules.remove_stop_words(df, stop_words)

In [49]:
#after 2
df.shape #3353 > #2746

(300, 2746)

In [37]:
#1
group_ls = [0,7,8,10,14,26,33,34,36,40,42,43,44,45,48,49,52,54,55,56,74,75,78,82,86,87,88,91,93,95]
stop_words = modules.make_stop_words_ls(km_df, group_ls)
df = modules.remove_stop_words(df, stop_words)

In [39]:
#after 1
df.shape #4436 > #3353

(300, 3353)

# 장소별 형용사 매트릭스 만들기

In [75]:
matrix = km_df.copy()

In [72]:
reviews = pd.read_csv('../../data/all_places_reviews.csv')

In [73]:
reviews

,p_id,review_kr,review_en
0,f239,최고예요. 편안하게 있다 갔어요. 분위기 작살!!. 분위기죠아요 연남동에서 유일하게...,It's the best. It's comfortable. He's gone. Th...
1,b209,희수야효선왕돈까스 쌍문1줄번게,Heesuya Hyosun king pork cutlet Ssangmun No.1
2,e810,더홀 했는데... 좀 부실해요. 가지마세요 상자 자물쇠도 안채워놓는 세팅실수해놓아서...,It was a little duhol. It's a little weak. Don...
3,a211,신나는 분위기에서 한잔하며 볼링 즐길 수 있는 곳. 좋은 분위기 속에서 맛있는 칵테...,A place where you can drink and enjoy bowling ...
4,b44,걷기 길찾기 하면 배수로로 안내하니 괜히 헤매지말고 바로 노을공원입구로 가시오. 풍...,"If you find a way to walk, you will be guided ..."
5,f206,좀 오래 전에 갔었는데 위치가 좀 애매해서 그렇지 식사류 너무 맛있고 좋아요. 손님...,"I've been there a while ago, but the location ..."
6,f20,보드게임 종류가 일단 많고 매장도 넓어요 직원분들이 게임 추천도 해주시고 룰도 친절...,There are many kinds of board games. There are...
7,g112,신림에서 발견한 가성비 갑 노래방 4곡에 천원 가기 전에 1000원짜리 지폐와 물...,"It is convenient to bring a 1,000 won bill and..."
8,f1913,최고예요,It's the best.
9,b88,완전 별루. 입장료도 비쌈. 비추,That's not a good idea. The entrance fee is al...


In [91]:
len(km_df.columns)

20

In [113]:
'active' in list(km_df['group_14'])

True

In [126]:
list(km_df['group_14'])

['different',
 'active',
 'available',
 'possible',
 'secret',
 'designated',
 'limited',
 'mandatory',
 'fit',
 'representative',
 'known',
 'recoverable',
 'present',
 'eligible',
 'preferred',
 'sole',
 'arranged',
 'key',
 'restricted',
 'discounted',
 'recommended',
 'fabricated',
 'wellknown',
 'respective',
 'developed',
 'unused',
 'purpose',
 'describe',
 'speaks',
 'applicable',
 'specified',
 'exist',
 'cited',
 'prearranged',
 'primary',
 'affiliated',
 'exempt',
 'used',
 'sponsored',
 'regardless',
 'exclusive',
 'standby',
 'understood',
 'utilized',
 'working',
 'negotiation',
 'prerequisite',
 'controlled',
 'responsible',
 'reserved',
 'affect',
 'subordinate',
 'valid',
 'dependent',
 'prominent',
 'established',
 'reachable',
 'vary',
 'purified',
 'prohibited',
 'extrapolated',
 'assigned',
 'registered',
 'translated',
 'selfemployed',
 'chartered',
 'contaminated',
 'packaged',
 'payable',
 'checkable',
 'attachment',
 'referenced',
 'chosen',
 'favored',
 'forbi

In [145]:
reviews

,p_id,review_kr,review_en
0,f239,최고예요. 편안하게 있다 갔어요. 분위기 작살!!. 분위기죠아요 연남동에서 유일하게...,It's the best. It's comfortable. He's gone. Th...
1,b209,희수야효선왕돈까스 쌍문1줄번게,Heesuya Hyosun king pork cutlet Ssangmun No.1
2,e810,더홀 했는데... 좀 부실해요. 가지마세요 상자 자물쇠도 안채워놓는 세팅실수해놓아서...,It was a little duhol. It's a little weak. Don...
3,a211,신나는 분위기에서 한잔하며 볼링 즐길 수 있는 곳. 좋은 분위기 속에서 맛있는 칵테...,A place where you can drink and enjoy bowling ...
4,b44,걷기 길찾기 하면 배수로로 안내하니 괜히 헤매지말고 바로 노을공원입구로 가시오. 풍...,"If you find a way to walk, you will be guided ..."
5,f206,좀 오래 전에 갔었는데 위치가 좀 애매해서 그렇지 식사류 너무 맛있고 좋아요. 손님...,"I've been there a while ago, but the location ..."
6,f20,보드게임 종류가 일단 많고 매장도 넓어요 직원분들이 게임 추천도 해주시고 룰도 친절...,There are many kinds of board games. There are...
7,g112,신림에서 발견한 가성비 갑 노래방 4곡에 천원 가기 전에 1000원짜리 지폐와 물...,"It is convenient to bring a 1,000 won bill and..."
8,f1913,최고예요,It's the best.
9,b88,완전 별루. 입장료도 비쌈. 비추,That's not a good idea. The entrance fee is al...


In [143]:
for i in range (0,len(reviews)):
    p_id = reviews['p_id'][i]
    review = reviews['review_en'][i]
    token_ls = nltk.word_tokenize(str(review));
    
    for j in range (0,cluster_count):
        count = 0
        col_name = 'group_'+str(j);
        for each_token in token_ls:
            for k in list(km_df[col_name]):
                if(str(each_token) == k):
                    count += 1
        matrix[col_name][i] = count;

In [144]:
matrix

,group_5,group_0,group_1,group_2,group_3,group_4,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,2,0,2,0,0,0,0,1,0,0,0
3,2,4,15,1,0,0,5,13,0,21,0,3,1,1,2,12,7,0,0,3
4,0,0,1,0,0,0,0,0,3,1,0,0,0,0,0,0,0,1,0,0
5,1,0,3,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
7,0,1,1,1,0,0,1,3,0,1,0,0,2,0,0,2,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [135]:
reviews['review_en'][3]

"A place where you can drink and enjoy bowling in an exciting atmosphere. I enjoyed bowling with delicious cocktails in a good atmosphere! It's not cheap, but you can enjoy it as much as the price! I was stressed out at work and went with my co-workers, but all my stress went away. I want to be better at bowling. The atmosphere was good and there were many things to enjoy besides bowling. If you are waiting in the back, you should wait again. Based on one drink per person, you can't even talk if it's cheap, but the cheapest coke was 5,900 won. The employees are kind, but they need a change. It was my first time to see a bowling alley selling food and cocktails, and I was surprised that the quality was better than I thought. Would it be perfect to have a birthday party with your friends? The employees are kind too! Unkindness. I went to Smashing Ball for a company dinner. It was a great experience. Bowling shoes were also a wing tip; I was satisfied with the quality and the staff servic

In [136]:
for i in range (20, 50):
    name = 'group_'+str(i)
    del matrix[name]

In [138]:
matrix

,group_5,group_0,group_1,group_2,group_3,group_4,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,2,0,2,0,0,0,0,1,0,0,0
3,2,4,15,1,0,0,5,13,0,21,0,3,1,1,2,12,7,0,0,3
4,0,0,1,0,0,0,0,0,3,1,0,0,0,0,0,0,0,1,0,0
5,1,0,3,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
7,0,1,1,1,0,0,1,3,0,1,0,0,2,0,0,2,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
